In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime, time
import pandas_datareader.data as pdr
import yfinance as yf 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [3]:
def stock_data(ticker, start):
    try:
        prices = yf.download(ticker, start, auto_adjust=True)['Close']
    except Exception as e:
        print(f"Error: {e}")
    return prices
    

In [124]:
def dual_momemtum(prices, stocks, market, mom_lookback=6, ma_window = 10, top_n=5):
    """Calculating 6 month dual momentum with monthly rebalancing"""


     #Monthly rebalancing to trade monthly
    prices_rb = prices.resample("ME").last()
    prices_market = prices[market]
   

    #Absolute Momemtum for the market
    abs_mom = prices_rb[market].pct_change(mom_lookback)
    trend = prices_rb[market] > prices_rb[market].rolling(ma_window).mean()
    market_signal = (abs_mom > 0) & trend

    #print(market_signal.mean()) #debug

    #Relative momentum for stocks
    stocks_return = prices_rb[stocks].pct_change(mom_lookback)

    #Ranking stocks
    ranks = stocks_return.rank(axis = 1, ascending= False, method='first')
    top_stocks = ranks <= top_n

   
    #monthly weights
    weights_rb = pd.DataFrame(0.0, index= prices_rb.index, columns=prices.columns)
    "this step is initializing weight matrix. rows are rebalance dates . Cash is represented as all zeros"

    for date in prices_rb.index:
        if market_signal.loc[date]:
            selected = top_stocks.loc[date]
            if selected.sum() > 0:
                weights_rb.loc[date, stocks] = (selected / selected.sum()) #assigning equal weight to selected stocks

    #Daily tradable weights
    weights = ( weights_rb.reindex(prices.index).ffill().shift(1).fillna(0))

    """  reindex -> expands weights from monthly to daily and add Nads to non-rebalace days.
    ffill() - forward-fills last rebelance weights and portfolio is held constant between rebalance dates
    shift(1) - signals use today's close price (prevent lookahead bias)
    fillna(0) - before first signal. everything is in cash which is presented by 0.  """

    #Strategy returns
    returns = prices.pct_change()
    strategy_returns =( weights * returns).sum(axis = 1)
    equity_curve = (1+ strategy_returns).cumprod()

    #print(weights.sum(axis=1).mean()) #debug
    #print(strategy_returns.std()) #debug

    #Market return
    market_return = prices_market.pct_change().dropna()


    "Performance metrics"
    #Annualised return - cagr -> compounded yearly growth rate of the startegy
    trading_days = 252
    cagr = ( (1+ + strategy_returns).prod()** (trading_days / len(strategy_returns)) -1 )
    cagr_market = ((1+market_return).prod()** (trading_days / len(market_return))-1 )

    #Annulised Volatility
    volatilty = strategy_returns.std() * np.sqrt(trading_days)

    #Sharpe Ratio
    sharpe_ratio = ( strategy_returns.mean() / strategy_returns.std()) * np.sqrt(trading_days)

    #maximum Drawdown
    rolling_max = equity_curve.cummax()
    drawdown = (equity_curve / rolling_max )-1
    max_drawdown = drawdown.min()

    #Calmer ratio -> return per unit drawdown
    calmer = cagr/abs(max_drawdown)

    win_rate = (strategy_returns >0).mean()

    turnover = weights.diff().abs().sum(axis=1).mean()

    #Summary table for performance

    performance = pd.Series( { 
        "CAGR" : cagr,
        "Market CAGR": cagr_market,
        "Volatility" : volatilty,
        "Sharpe Ratio" : sharpe_ratio,
        "max Drawdown" : max_drawdown,
        "Calmer Ratio" : calmer,
        "Win rate" : win_rate,
        "Turnover": turnover
    })
    print("Performace:\n", performance)


    #Plot Ranks
    fig1 = go.Figure()

    for col in ranks.columns:
        fig1.add_trace(go.Scatter(x=ranks.index,
                                 y=ranks[col],
                                 mode="lines", name=col))
        
    
    fig1.update_layout( height=800, width =1000, autosize=False,
        title="Momentum Rank", xaxis_title='Date', yaxis_title="Rank(1=Best)", yaxis_autorange="reversed",
        template ="plotly_white", hovermode="x unified")
    
    fig1.add_hrect(y0=0.5, y1=top_n+0.5, fillcolor="green", opacity=0.15,
                   layer="below", line_width=0)
    
    fig1.show()



    #Equity curve and Market regime
    
    
    market_signal_daily = (market_signal.reindex(prices_market.index).ffill().fillna(False))
    
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve, name='Equity curve', line=dict(width=2)))

    risk_on = market_signal_daily
    start = None

    for date, val in risk_on.items():
        if val and start is None:
            start = date
        elif not val and start is not None:
            fig.add_vrect(x0=start, x1=date,
                          fillcolor ="green", opacity=0.15, layer="below", line_width=0)
            start = None
    
    if start is not None:
        fig.add_vrect(x0=start, x1=risk_on.index[-1],
                      fillcolor='green', opacity = 0.15, layer="below", line_width=0)



    fig.update_layout(
    title="Equity curve with Market regime (Risk-on Shaded)",
    template="plotly_white", hovermode = "x unified"
                )

    fig.show()
    

    return equity_curve

In [97]:
stocks = ['LLY', 'JNJ', 'AMGN','MRK','NVO','PFE']
market ='SPY'
ticker = stocks + [market]

In [98]:
prices = stock_data(ticker, "2010-01-01")
prices.tail(10)

[*********************100%***********************]  7 of 7 completed


Ticker,AMGN,JNJ,LLY,MRK,NVO,PFE,SPY
Date,,,,,,,
2025-12-15,325.309998,214.169998,1062.189941,100.260002,50.369999,26.430000,678.724426
2025-12-16,326.739990,209.300003,1054.290039,98.269997,48.959999,25.530001,676.869934
2025-12-17,326.010010,210.330002,1041.790039,99.180000,47.770000,25.040001,669.421936
2025-12-18,324.420013,208.309998,1056.880005,100.690002,47.610001,25.040001,674.476929
2025-12-19,327.380005,206.369995,1071.439941,101.089996,48.090000,25.190001,680.590027
2025-12-22,331.390015,207.320007,1076.479980,104.720001,48.099998,25.209999,684.830017
2025-12-23,331.489990,205.779999,1071.640015,105.040001,51.610001,24.879999,687.960022
2025-12-24,333.959991,207.779999,1076.979980,106.449997,52.560001,25.030001,690.380005
2025-12-26,332.929993,207.630005,1077.750000,106.779999,52.400002,25.090000,690.309998


In [125]:
equity_curve = dual_momemtum(prices,stocks,market,mom_lookback=6, ma_window=10, top_n=4)
equity_curve

Performace:
 CAGR            0.169913
Market CAGR     0.080827
Volatility      0.254092
Sharpe Ratio    0.745037
max Drawdown   -0.444659
Calmer Ratio    0.382121
Win rate        0.358880
Turnover        0.004819
dtype: float64


/var/folders/dw/c3ntnt1x4kn59s38pldw62hr0000gn/T/ipykernel_91273/1628586319.py:117: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Date
2000-01-03     1.000000
2000-01-04     1.000000
2000-01-05     1.000000
2000-01-06     1.000000
2000-01-07     1.000000
                ...    
2025-12-22    57.685272
2025-12-23    58.497251
2025-12-24    58.551049
2025-12-26    58.861644
2025-12-29    58.605746
Length: 6537, dtype: float64

In [73]:
def plot_return(equity_curve):
    """Plot the equity curve and drawdown"""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve, mode="lines", name='Dual Momentum'))

    fig.update_layout(title = "Dual Momentum Equity curve",
                      xaxis_title="Date", yaxis_title ='Portfolio value', yaxis_type = "log",
                      template="plotly_white", hovermode ='x unified')
    fig.show()

    rolling_max = equity_curve.cummax()
    drawdown = (equity_curve / rolling_max )-1

    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x = drawdown.index, y = drawdown, fill ='tozeroy', name = 'Drawdown'))
    fig1.update_layout(title='Drawdown', 
                       xaxis_title = 'Date', yaxis_title = 'Drawdown', template='plotly_white',
                       hovermode ='x unified')
    fig1.show()




Tech Stocks

In [126]:
stocks = ['AMD', 'TSM', "NVDA",  'GOOG']
market ='SPY'
ticker = stocks + [market]


In [127]:
prices = stock_data(ticker, "2000-01-01")
prices.tail(10)

[*********************100%***********************]  5 of 5 completed


Ticker,AMD,GOOG,NVDA,SPY,TSM
Date,,,,,
2025-12-16,209.169998,307.730011,177.720001,676.869934,286.869995
2025-12-17,198.110001,298.059998,170.940002,669.421936,276.959991
2025-12-18,201.059998,303.750000,174.139999,674.476929,284.679993
2025-12-19,213.429993,308.609985,180.990005,680.590027,288.950012
2025-12-22,214.949997,311.329987,183.690002,684.830017,293.279999
2025-12-23,214.899994,315.679993,189.210007,687.960022,296.950012
2025-12-24,215.039993,315.670013,188.610001,690.380005,298.799988
2025-12-26,214.990005,314.959991,190.529999,690.309998,302.839996
2025-12-29,215.610001,314.390015,188.220001,687.849976,300.920013


In [128]:
equity_curve = dual_momemtum(prices, stocks, market, mom_lookback=6, ma_window=10, top_n= 3)

Performace:
 CAGR            0.210468
Market CAGR     0.080822
Volatility      0.275359
Sharpe Ratio    0.831453
max Drawdown   -0.458725
Calmer Ratio    0.458812
Win rate        0.361884
Turnover        0.008106
dtype: float64


/var/folders/dw/c3ntnt1x4kn59s38pldw62hr0000gn/T/ipykernel_91273/1628586319.py:44: FutureWarning:

The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



/var/folders/dw/c3ntnt1x4kn59s38pldw62hr0000gn/T/ipykernel_91273/1628586319.py:117: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [129]:
plot_return(equity_curve)